# Running the simulations
In this notebook, you will run the simulations needed for plotting the figures of the article, found in a second notebook. You can choose to run them on multiple cores (default) or on a computing cluster. For further explanations considering the execution framework, see https://github.com/wschuell/experiment_manager

In [1]:
import naminggamesal as ngal
import experiment_manager as xp_man
from experiment_manager.batchexp import BatchExp

from IPython.display import clear_output
import time

Defining the basic configuration skeleton of a simulation:

In [2]:
def xp_cfg(N,M,W,time_scale=0,active=False,gamma=0.1):
    base_cfg = {
          "step": "log_improved", 
          "pop_cfg": {
            "voc_cfg": {
          "voc_type": 'matrix_new'
            }, 
        "strat_cfg": {
          "vu_cfg": {
            "vu_type": "minimal"
          }, 
          "success_cfg": {
            "success_type": "global_norandom"
              }, 
              "strat_type": "naive" #random topic choice: changed if active is True
            }, 
            "nbagent": N, #population size
            "env_cfg": {
              "env_type": "simple", 
              "M": M, #number of meanings
              "W": W  #number of words
            }, 
            "interact_cfg": {
              "interact_type": "speakerschoice"
            }
          }
        }
    if active:
        base_cfg['pop_cfg']['strat_cfg'].update(**{
                'strat_type':'lapsmax_mab_explothreshold','bandit_type':'bandit_laps','gamma':gamma,'time_scale':time_scale,
                "memory_policies": [{
                      "time_scale": time_scale, 
                      "mem_type": "interaction_counts_sliding_window_local"
                        }], })
    return base_cfg

We define here the parameters of what we want to run. Different base configurations (testcases), several values of time scale parameter $\tau$, and monitoring functions. nb_iter is the number of trials per configuration.

In [3]:

testcases = [
    {'N':40,'M':40,'W':40,'Tmax':10*10**4,'gamma':0.01},  #N: population size, M:number of meanings
    {'N':40,'M':40,'W':40,'Tmax':10*10**4,'gamma':1.}     #W: number of words, Tmax: #interactions per run
]

timesc_values = [1,2,3,4,5,6,7,8,9,10,20,30,50] #values for the timescale parameter

analysis_f = [  #analysis functions
    'srtheo',     # probability of success
    'Nlink',      # vocabulary size (agent level): local complexity
    'N_d',        # vocabulary size (population level): global complexity
    'homonymy',   # average degree of homonymy 
    'synonymy',   # average degree of synonymy
    'N_meanings', # average number of meanings known by each agent
    'N_words'     # average number of words known by each agent
]

nb_iter = 8 # results averaged over nb_iter trials per configuration

Define database and available execution configurations

In [4]:
db = ngal.ngdb.NamingGamesDB(do_not_close=True)

#config for local execution (single core)
jq_cfg_local = {'jq_type':'local','erase':True}#_multiprocess'}

#config for local execution on multiple cores
jq_cfg_local_multi = {'jq_type':'local_multiprocess','erase':True}

#example config for execution on a computing cluster (here PlaFrim experimental testbed)
jq_cfg_plafrim = {'jq_type':'plafrim','erase':True,
         'modules':['language/python/3.5.2'],
         'virtual_env':'testvenv',
'requirements' : [
    '-e git+https://github.com/wschuell/experiment_manager.git@origin/develop#egg=experiment_manager',
    '-e git+https://github.com/flowersteam/naminggamesal.git@origin/develop#egg=naminggamesal'
]}

The batch object will be the one executing our simulations. You can choose the execution configuration, among those defined above.

In [5]:
batch = BatchExp(name='cogsci_local_multi',db=db,estimated_time=1200,jq_cfg=jq_cfg_local_multi)
#batch = BatchExp(name='cogsci_local',db=db,estimated_time=1200,jq_cfg=jq_cfg_local)
#batch = BatchExp(name='cogsci_plafrim',db=db,estimated_time=1200,jq_cfg=jq_cfg_plafrim)

With the following function, we can generate all simulation configurations, to be fed to the batch object before actual computation.

In [6]:
def gen_cfg(N,M,W,Tmax,gamma=0.1):
    xp_cfg_l = [xp_cfg(N,M,W)]+[xp_cfg(N,M,W,active=True,time_scale=t,gamma=gamma) for t in timesc_values]
    ans = []
    for cfg in xp_cfg_l:
        ans.append({'xp_cfg':cfg,'method':analysis_f,'nb_iter':nb_iter,'tmax':Tmax})
    return ans

Adding all simulation configurations to batch object

In [7]:
for testc in testcases:
    print(str(testc))
    batch.add_jobs(gen_cfg(**testc))

{'N': 40, 'M': 40, 'W': 40, 'Tmax': 100000, 'gamma': 0.01}
{'N': 40, 'M': 40, 'W': 40, 'Tmax': 100000, 'gamma': 1.0}


Running all experiments (can be really long! Up to a couple of days on a single core)

In [8]:
while batch.jobqueue.job_list:
    clear_output()
    batch.update_queue()
    time.sleep(10)
clear_output()
batch.auto_finish_queue(t=10,coeff=2)

[2018 05 14 01:45:02]: Queue updated
    total: 0
    

    execution time: 9 h 15 min 20.924087524414062 s
    jobs done: 253
    jobs restarted: 0
    jobs extended: 0

